In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/MyDrive/Trendyol.zip

Archive:  /content/gdrive/MyDrive/Trendyol.zip
  inflating: supplier_return.csv     
  inflating: test_data.csv           
  inflating: transactions.csv        
  inflating: user_demographics.csv   
  inflating: products.csv            
  inflating: qa.csv                  
  inflating: reviews.csv             
  inflating: supplier_defective_return.csv  
  inflating: supplier_disputed_return.csv  


In [ ]:
products = pd.read_csv(r"products.csv")
qa = pd.read_csv(r"qa.csv")
reviews = pd.read_csv(r"reviews.csv")
supplier_defective_return = pd.read_csv(r"supplier_defective_return.csv")
supplier_disputed_return = pd.read_csv(r"supplier_disputed_return.csv")
supplier_return = pd.read_csv(r"supplier_return.csv")
test_data = pd.read_csv(r"test_data.csv")
transactions = pd.read_csv(r"transactions.csv")
user_demographics = pd.read_csv(r"user_demographics.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
transactions[["product_content_id","user_id","order_parent_id"]] = transactions[["product_content_id","user_id","order_parent_id"]].drop_duplicates()

In [ ]:
new = test_data["id"].str.split("|", n = 2, expand = True)

In [ ]:
test_data["user_id"]= new[0]
test_data["product_content_id"]= new[1]
test_data["order_parent_id"]= new[2]
test_data = test_data.drop(["id"],axis=1)


In [ ]:
test_data["user_id"] = test_data["user_id"].astype(int)
test_data["product_content_id"] = test_data["product_content_id"].astype(int)
test_data["order_parent_id"] = test_data["order_parent_id"].astype(int)

In [ ]:
test_transactions = pd.merge(test_data,transactions,how="left",on=["product_content_id","user_id","order_parent_id"])
test_transactions.shape



(7471, 19)

In [ ]:
test_transactions[test_transactions["order_parent_id"]==725291348]

,expected,user_id,product_content_id,order_parent_id,order_date,is_elite_user,supplier_id,order_line_item_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned
7468,NaN,71115350,94438742,725291348,2021-08-03 15:51:18.124000+00:00,0,225138,1113459138,200752396,229.90,143.92,11.330000,NaN,0.0,NaN,NaN,0,True,NaN
7469,NaN,71115350,100017676,725291348,2021-08-03 15:51:18.124000+00:00,0,968,1113459117,166171623,39.99,27.99,1.342857,NaN,0.0,Trendyolmilla&Trendyol Man&Trendyol Modest-Kaç...,30.0,0,True,NaN


In [ ]:
test_products = pd.merge(test_transactions,products,how="left",on=["product_variant_id", "product_content_id"])

In [ ]:
reviews = reviews.groupby(["product_content_id", "supplier_id"])["rate"].mean().to_frame()
test_review = pd.merge(test_products,reviews,how="left",on=["product_content_id", "supplier_id"])
del reviews


In [ ]:
user_id_gender = user_demographics.drop(["birth_date", "membership_date"], axis=1)
test_demographics = pd.merge(test_review,user_id_gender,how="left",on="user_id")
del user_id_gender
del user_demographics


In [ ]:
test_return = pd.merge(test_demographics,supplier_return,how="left",on = "supplier_id")
 

In [ ]:
supplier_defective_return = supplier_defective_return.rename(columns = {"returnRate":"defective_returnRate"})
test_defective_return = pd.merge(test_return,supplier_defective_return,how="left",on = "supplier_id")


In [ ]:
del supplier_disputed_return["unresolved_claim"]
del supplier_disputed_return["unresolved_accepted_claim"]


In [ ]:
test_disputed_return = pd.merge(test_defective_return,supplier_disputed_return,how="left",on = "supplier_id")


In [ ]:
df = test_disputed_return 
df.head()


,expected,user_id,product_content_id,order_parent_id,order_date,is_elite_user,supplier_id,order_line_item_id,product_variant_id,original_price,discounted_price,ship_cost,coupon_id,coupon_discount,promotion_name,promotion_award_value,is_wallet_trx,is_saved_card_trx,is_returned,product_id,product_name,brand_id,brand_name,gender_id,gender_name,category_id,category_name,color_id,color_name,supplier_color_name,attributet_name,attribute_value,rate,gender,returnRate,defective_returnRate,total_claim,unresolvedclaim_percentage
0,NaN,18115,87073974,726348863,2021-08-04 16:03:34.498000+00:00,1,968,1115248748,145716801,89.99,53.99,9.400000,NaN,0.0,NaN,NaN,1,False,NaN,31684775,Yeşil Yıldızlı Dokuma Pijama Takımı THMSS20PT0295,40,TRENDYOLMİLLA,4295.0,Kadın / Kız,563,Pijama Takımı,7012.0,Yeşil,Yeşil,BEDEN,38,4.486339,Bayan,0.127110,0.002828,3269293,0.000000
1,NaN,27588,118753298,723169934,2021-08-01 22:32:11.468000+00:00,0,968,1109728166,191196258,113.95,91.16,1.044444,NaN,0.0,NaN,NaN,0,True,NaN,94802386,Kadın Yeşil Kolsuz Keten Viskon Çan Elbise JH0...,8727,Happiness İst.,4295.0,Kadın / Kız,1182,Elbise,7012.0,Yeşil,Yeşil,BEDEN,36,4.310345,Bayan,0.127110,0.002828,3269293,0.000000
2,NaN,29165,35699988,724122443,2021-08-02 21:39:28.579000+00:00,1,1188,1111326869,63808285,119.99,83.99,8.880000,NaN,0.0,Defacto sepet 30 - Sepette %30 İndirim,30.0,0,False,NaN,29982007,Kız Çocuk Tül Detaylı Elbise,3268,DeFacto,4295.0,Kadın / Kız,1182,Elbise,6998.0,Beyaz,Beyaz-WT34,BEDEN,8-9 Yaş,4.412698,Bayan,0.057645,0.000978,328876,0.264478
3,NaN,29969,5893187,722342150,2021-08-01 01:07:16.583000+00:00,1,968,1108280452,19449310,59.99,50.99,4.700000,NaN,0.0,NaN,NaN,0,False,NaN,5047604,Kiremit Düğme Detaylı Crop Bluz TWOSS19EH0184,40,TRENDYOLMİLLA,4295.0,Kadın / Kız,590,Bluz,7011.0,Turuncu,Kiremit,BEDEN,36,4.291262,Bayan,0.127110,0.002828,3269293,0.000000
4,NaN,35107,93897418,723421057,2021-08-02 10:54:53.842000+00:00,1,968,1110143484,156456704,69.99,59.49,4.700000,NaN,0.0,Trendyolmilla&Trendyol Man&Trendyol Modest Yen...,15.0,0,False,NaN,5255654,Haki Kemer Detaylı Şort & Bermuda TWOSS19IE0062,40,TRENDYOLMİLLA,4295.0,Kadın / Kız,530,Şort & Bermuda,7015.0,Haki,Haki,BEDEN,42,4.425595,Bayan,0.127110,0.002828,3269293,0.000000


In [ ]:
df['AVG_spend'] = df['original_price'].groupby(df["user_id"]).transform('mean') ## Added a column that shows average amount of money each user spend.
df["parent_product_user"] = df.user_id.map(str) + "|" + df.order_parent_id.map(str) + "|"+ df.product_content_id.map(str)
df["count"] = df.groupby(["parent_product_user"])["product_variant_id"].transform("count") # Added a column that shows how many units the user bought of different variants of the same product.

In [ ]:
del df["order_date"]
del df["user_id"]
del df["supplier_id"]
del df["order_line_item_id"]
del df["order_parent_id"]
del df["product_content_id"]
del df["product_variant_id"]
del df["coupon_id"]
del df["promotion_name"]
del df["product_id"]
del df["product_name"]
del df["brand_id"]
del df["brand_name"] # Sonra bakılcak.
del df["gender_id"]
del df["category_id"]
del df["category_name"] # Sonra bakılcak.
del df["color_id"]
del df["supplier_color_name"] 
del df["attributet_name"] # Sonra Bakılcak.
del df["attribute_value"]


In [ ]:
df["ship_cost"] = df["ship_cost"].fillna(0)

df["promotion_award_value"] = df["promotion_award_value"].fillna(0)

df["rate"] = df["rate"].fillna(0)

df["color_name"] = df["color_name"].fillna("yok")

df["unresolvedclaim_percentage"] = df["unresolvedclaim_percentage"].fillna(0)

df["total_claim"] = df["total_claim"].fillna(0)

df["defective_returnRate"] = df["defective_returnRate"].fillna(0)

df["returnRate"] = df["returnRate"].fillna(0)



In [ ]:
df = df.drop(["is_returned","expected"],axis=1)


In [ ]:
pip install category_encoders

     |████████████████████████████████| 82 kB 484 kB/s 


In [ ]:
import category_encoders as ce
ohe = ce.OneHotEncoder(handle_unknown='value', use_cat_names=True)

OH_cols_train = pd.DataFrame(ohe.fit_transform(df[["gender_name", "color_name", "gender"]])) #One-Hot encoding applied to train dataset
OH_cols_train.index = df.index
df = df.drop(columns = ["gender_name", "color_name", "gender"]) #Dropping categorical columns
df = pd.concat([df,OH_cols_train],axis = 1) # Assembling df with encoded columns


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df["is_saved_card_trx"] = df["is_saved_card_trx"].astype(int)

In [ ]:
df["gender_name_Kadın"] = np.zeros(np.shape(df.index))
df["gender_name_Unknown"] = np.zeros(np.shape(df.index))
df["color_name_Altın"] = np.zeros(np.shape(df.index))
df["color_name_yok"] = np.zeros(np.shape(df.index))
df["color_name_Metalik"] = np.zeros(np.shape(df.index))
df = df[['is_elite_user', 'original_price', 'discounted_price', 'ship_cost',
       'coupon_discount', 'promotion_award_value', 'is_wallet_trx',
       'is_saved_card_trx', 'rate', 'returnRate', 'defective_returnRate',
       'total_claim', 'unresolvedclaim_percentage', 'AVG_spend', 'count',
       'gender_name_Erkek', 'gender_name_Kadın / Kız', 'gender_name_Unisex',
       'gender_name_Kadın', 'gender_name_Unknown', 'color_name_Beyaz',
       'color_name_Yeşil', 'color_name_Gri', 'color_name_Mor',
       'color_name_Mavi', 'color_name_Haki', 'color_name_Siyah',
       'color_name_Bej', 'color_name_Pembe', 'color_name_Kırmızı',
       'color_name_Turuncu', 'color_name_Sarı', 'color_name_Kahverengi',
       'color_name_Ekru', 'color_name_Lacivert', 'color_name_Bordo',
       'color_name_Turkuaz', 'color_name_Gümüş', 'color_name_Metalik',
       'color_name_Altın', 'color_name_yok', 'gender_Bayan', 'gender_Bay',
       'gender_UNKNOWN', 'gender_kvkktalepsilindi']]

In [ ]:
import pickle
# Load the Model back from file
with open("LGBM_TUNED_2.pk1", 'rb') as file:  
    clf = pickle.load(file)



In [ ]:
ypred = clf.predict(df)

In [ ]:
test_data = pd.read_csv(r"test_data.csv")

In [ ]:
test_data["expected"] = ypred
test_data

,id,expected
0,18115|87073974|726348863,1.0
1,27588|118753298|723169934,1.0
2,29165|35699988|724122443,0.0
3,29969|5893187|722342150,1.0
4,35107|93897418|723421057,0.0
...,...,...
7466,71083899|100019570|728638514,0.0
7467,71106534|35624085|725372928,0.0
7468,71115350|94438742|725291348,0.0
7469,71115350|100017676|725291348,0.0


In [ ]:
test_data["expected"] = test_data["expected"].astype(int)

In [ ]:
test_data.set_index("id").to_csv('submission_lgbm_F2.csv')